#### Install Dependencies

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@1f52468"


In [ ]:
#import torch
#if torch.cuda.get_device_capability()[0] >= 8:
#    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-n_ibvp0y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-n_ibvp0y
  Resolved https://github.com/huggingface/transformers to commit b53e44e847230a39caecc320527776ed80a3ceff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10057815 sha256=f7e38df1fa28041dd5e6af11b90b66a15ee94e64f32c4ad7a8808e40cbb11895
  Stored in directory: /tmp/pip-ephem-wheel-cache-w3jpi93e/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully 

#### Mount Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/mistral_unsloth_4

Mounted at /content/drive
/content/drive/MyDrive/mistral_unsloth_4


#### Path

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None
load_in_4bit = True

# Load the fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Mistral-Nemo-Instruct-lro-finetune",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# Prepare the model for inference
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Mistral patching. Transformers = 4.47.0.dev0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(

#### Data Preparation

In [ ]:
import pandas as pd
from datasets import Dataset

data_type='Synthetic'

# Load Data
df = pd.read_excel('LR_Dataset_Original_Sythetic_Final.xlsx')
if data_type=='Original':
 df = df[df['Source'] == 'Original'] # Only original

df= df[['Sentence','Category' ,'Classification']]

df['Sentence'] = df['Sentence'].str.capitalize()
df=df.sample(frac=1).reset_index(drop=True)
df['Sentence'] = (df['Sentence']
                  .str.strip()
                  .str.replace(r'\n|\r', ' ', regex=True)
                  .str.replace(r'\s{2,}', ' ', regex=True))
df


,Sentence,Category,Classification
0,"However, to the best of our knowledge, no stud...",RESEARCH GAP,VALIDATION
1,By employing cutting-edge interactive visualiz...,EXTENSION,TRAINING
2,Assess the veracity of the following statement...,OTHER,TRAINING
3,"In their work, they suggested a method named t...",DESCRIPTION,TRAINING
4,The goal of session-based recommendation is to...,OVERALL,TRAINING
...,...,...,...
2935,The evidence is inconclusive regarding whether...,RESEARCH GAP,TRAINING
2936,The challenge lies in determining if survey qu...,RESEARCH GAP,TRAINING
2937,Ekstrand and kluver [12] investigated how coll...,DESCRIPTION,TRAINING
2938,"According to the source [12], the cit techniqu...",DESCRIPTION,TRAINING


In [ ]:
test_df = df[df['Classification'] == 'TEST']
#test_df =test_df.shuffle(seed=42)



In [ ]:
print(test_df.columns)

Index(['Sentence', 'Category', 'Classification'], dtype='object')


In [ ]:
print(test_df.head(1))

                                             Sentence     Category  \
66  Mcauley et al. [21] mine key attributes from t...  DESCRIPTION   

   Classification  
66           TEST  


In [ ]:
def create_conversation_test(row):
    sentence = row['Sentence']


    human = (
                "You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\n"
                "Categories:\n "
                "1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n "
                "2. RESEARCH GAP: Highlights the need for further research within the topic.\n"
                "3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n"
                "4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “found”, “demonstrated”, and “observed” or phrases like “the findings indicate”.\n"
                "5. LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study that hinders generalizability or reliability in a previous study.\n"
                "6. EXTENSION: Describes how the current study addresses or extends previous studies by stating the overall idea, contrasting ideas or elaborating further ideas. It usually uses the words “we” or “our”.\n"
                "7. OTHER: Any text that does not fit the above categories.\n"
                "Procedure:\n"
                "1. Determine whether the subject of the setence is a topic or a study.\n"
                "2. Identify the most suitable category based on the content. Do not create new categories. Use the categories given above.\n"
                "3. Provide the category number that best fits the sentence. Just provide the category number without any explanation.\n"

                f"Sentence: {sentence}.\n"
            )


    return [
        {"from": "human", "value": human},

    ]


In [ ]:
test_df['conversations'] = test_df.apply(create_conversation_test, axis=1)

<ipython-input-10-f5d88970cb99>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['conversations'] = test_df.apply(create_conversation_test, axis=1)


In [ ]:
test_df['conversations']


,conversations
66,"[{'from': 'human', 'value': 'You are a researc..."
81,"[{'from': 'human', 'value': 'You are a researc..."
89,"[{'from': 'human', 'value': 'You are a researc..."
111,"[{'from': 'human', 'value': 'You are a researc..."
164,"[{'from': 'human', 'value': 'You are a researc..."
...,...
2900,"[{'from': 'human', 'value': 'You are a researc..."
2912,"[{'from': 'human', 'value': 'You are a researc..."
2924,"[{'from': 'human', 'value': 'You are a researc..."
2925,"[{'from': 'human', 'value': 'You are a researc..."


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [ ]:
from datasets import load_dataset

test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

#### Train the model

In [ ]:
test_df_converted = test_dataset.to_pandas()
test_df_converted

,Sentence,Category,Classification,conversations,__index_level_0__,text
0,Mcauley et al. [21] mine key attributes from t...,DESCRIPTION,TEST,"[{'from': 'human', 'value': 'You are a researc...",66,<s>[INST] You are a researcher that should ass...
1,"We also thank tal august, youngwook do, kennet...",OTHER,TEST,"[{'from': 'human', 'value': 'You are a researc...",81,<s>[INST] You are a researcher that should ass...
2,Tie [41] is a time-aware incremental embedding...,DESCRIPTION,TEST,"[{'from': 'human', 'value': 'You are a researc...",89,<s>[INST] You are a researcher that should ass...
3,Existing approaches for the task can be mainly...,OVERALL,TEST,"[{'from': 'human', 'value': 'You are a researc...",111,<s>[INST] You are a researcher that should ass...
4,"Advances in ml, largely in language models, ha...",OVERALL,TEST,"[{'from': 'human', 'value': 'You are a researc...",164,<s>[INST] You are a researcher that should ass...
...,...,...,...,...,...,...
135,While we have a well-balanced sample in terms ...,LIMITATION,TEST,"[{'from': 'human', 'value': 'You are a researc...",2900,<s>[INST] You are a researcher that should ass...
136,"Additionally, authors would like to give thank...",OTHER,TEST,"[{'from': 'human', 'value': 'You are a researc...",2912,<s>[INST] You are a researcher that should ass...
137,Experiments have found that electrical stimula...,RESULT,TEST,"[{'from': 'human', 'value': 'You are a researc...",2924,<s>[INST] You are a researcher that should ass...
138,Aequitas [41] is an auditing toolkit that meas...,DESCRIPTION,TEST,"[{'from': 'human', 'value': 'You are a researc...",2925,<s>[INST] You are a researcher that should ass...


In [ ]:
prompt=test_df_converted['text'].loc[123]
prompt


'<s>[INST] You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “

In [ ]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(prompt,
                return_tensors="pt"
            ).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

answer

'You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “found”, “d

In [ ]:
import re
def extract_text(text):
    # Extract text after "Classification: "
    match = re.search(r"Classification:\s*(.*)", text)
    if match:
        classification = match.group(1)
        # Remove extra spaces and convert to uppercase
        cleaned_classification = ' '.join(classification.split()).strip().upper()
        return cleaned_classification
    return ""

In [ ]:
extract_text(answer)

'DESCRIPTION'

In [ ]:
# Getting the Classification
def get_classification(data_point,model,tokenizer):
    """
    Gets the classification for a data point using the fine-tuned model.
    """
    FastLanguageModel.for_inference(model)

    inputs = tokenizer(
                data_point['text'],
                return_tensors="pt"
            ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(f"Sentence : {data_point.index[-1]}")
    print(answer)
    data_point['Prediction_Finetune']=answer
    data_point['Prediction_Finetune_Clean']=extract_text(answer)



    return data_point

# Apply the get_classification function to the dataset using map
test_df_converted = test_df_converted.apply(lambda row: get_classification(row, model, tokenizer), axis=1)

Sentence : text
You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.
Categories:
 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.
 2. RESEARCH GAP: Highlights the need for further research within the topic.
3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.
4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “

In [ ]:
test_df_converted

,Sentence,Category,Classification,conversations,__index_level_0__,text,Prediction_Finetune,Prediction_Finetune_Clean
0,Mcauley et al. [21] mine key attributes from t...,DESCRIPTION,TEST,"[{'from': 'human', 'value': 'You are a researc...",66,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,DESCRIPTION
1,"We also thank tal august, youngwook do, kennet...",OTHER,TEST,"[{'from': 'human', 'value': 'You are a researc...",81,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,OTHER
2,Tie [41] is a time-aware incremental embedding...,DESCRIPTION,TEST,"[{'from': 'human', 'value': 'You are a researc...",89,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,DESCRIPTION
3,Existing approaches for the task can be mainly...,OVERALL,TEST,"[{'from': 'human', 'value': 'You are a researc...",111,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,OVERALL
4,"Advances in ml, largely in language models, ha...",OVERALL,TEST,"[{'from': 'human', 'value': 'You are a researc...",164,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,OVERALL
...,...,...,...,...,...,...,...,...
135,While we have a well-balanced sample in terms ...,LIMITATION,TEST,"[{'from': 'human', 'value': 'You are a researc...",2900,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,LIMITATION
136,"Additionally, authors would like to give thank...",OTHER,TEST,"[{'from': 'human', 'value': 'You are a researc...",2912,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,OTHER
137,Experiments have found that electrical stimula...,RESULT,TEST,"[{'from': 'human', 'value': 'You are a researc...",2924,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,RESULT
138,Aequitas [41] is an auditing toolkit that meas...,DESCRIPTION,TEST,"[{'from': 'human', 'value': 'You are a researc...",2925,<s>[INST] You are a researcher that should ass...,You are a researcher that should assign a clas...,DESCRIPTION


In [ ]:
test_df_converted.to_csv('Mistral7B_Instruct_FT_Test_LoRA_Exp_70_Augmented.csv', index=False)
#test_df_converted.to_csv('Mistral7B_Based_FT_Test_NEFT_Augmented.csv', index=False)
#test_df_converted.to_csv('Mistral7B_Based_FT_Test_LoRA_Augmented_1.csv', index=False)


#### Make Inference